# 1. Kết nối tới drive , import một số thư viện cần thiết

In [89]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [90]:
import numpy.polynomial.polynomial as poly
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns


from datetime import datetime
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.linear_model import Lasso
from sklearn.utils import column_or_1d

from keras.models import Sequential
from keras.layers import Dense, GRU
from keras.regularizers import L1L2

In [91]:
VNM = pd.read_csv("/content/drive/MyDrive/Data pttk/Vinamilk.csv")

In [92]:
VNM

,Date,Close,Open,High,Low,Vol.,Change %
0,16/01/2017,99924.0,100319.0,100398.0,99845.0,723.16K,-0.32%
1,17/01/2017,99608.0,100082.0,100082.0,99608.0,698.86K,-0.32%
2,18/01/2017,99529.0,99529.0,99687.0,98897.0,1.02M,-0.08%
3,19/01/2017,99529.0,99766.0,99766.0,99450.0,1.04M,0.00%
4,20/01/2017,100319.0,99529.0,101346.0,99529.0,1.91M,0.79%
...,...,...,...,...,...,...,...
1724,11/12/2023,70000.0,70500.0,70500.0,69900.0,4.73M,1.45%
1725,12/12/2023,69400.0,70000.0,70100.0,69000.0,2.79M,-0.86%
1726,13/12/2023,68200.0,69600.0,69700.0,68100.0,4.51M,-1.73%
1727,14/12/2023,68400.0,68400.0,68900.0,68200.0,1.46M,0.29%


In [93]:
VNM.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1729 entries, 0 to 1728
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Date      1729 non-null   object 
 1   Close     1729 non-null   float64
 2   Open      1729 non-null   float64
 3   High      1729 non-null   float64
 4   Low       1729 non-null   float64
 5   Vol.      1729 non-null   object 
 6   Change %  1729 non-null   object 
dtypes: float64(4), object(3)
memory usage: 94.7+ KB


# 2. Tiền xử lý dữ liệu

### Loại bỏ một số dữ liệu trống trong dataset

In [94]:
# Drop missing values
VNM = VNM.dropna()

In [95]:
VNM1 = VNM.reset_index()['Close']

In [96]:
VNM['Index'] = np.arange(len(VNM.index))

<h1> 3.Scale dữ liệu

In [97]:
scaler=MinMaxScaler(feature_range=(0,1))
VNM1=scaler.fit_transform(np.array(VNM1).reshape(-1,1))

# 4. Chia tập dữ liệu để train-test-validate theo tỉ lệ 7:2:1

In [98]:
# Split the data into training, testing, and validation sets
train_size = int(0.7 * len(VNM1))
test_size = int(0.2 * len(VNM1))
val_size = len(VNM1) - train_size - test_size

In [99]:
train_data = VNM1[:train_size]
test_data = VNM1[train_size:train_size+test_size]
val_data = VNM1[train_size+test_size:]

<h1>Hàm Create Dataset

In [100]:
#Chuyển mảng giá trị của dataset thành ma trận
def create_dataset(dataset, time_step=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-time_step-1):
		a = dataset[i:(i+time_step), 0]   ###i=0, X=0,1,2,3-----99   Y=100
		dataX.append(a)
		dataY.append(dataset[i + time_step, 0])
	return numpy.array(dataX), numpy.array(dataY)

In [101]:
import numpy
#Reshape into X=t,t+1, t+2... t+99 and Y= t+100
time_step =100
X_train, y_train = create_dataset(train_data, time_step)
X_val, yval = create_dataset(val_data, time_step)
X_test, ytest = create_dataset(test_data, time_step)

In [102]:
#Reshape input to be [samples, time steps, features] which is required for GRU
X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)
X_val = X_val.reshape(X_val.shape[0],X_val.shape[1] , 1)

In [103]:
# Train a Lasso model
lasso = Lasso(alpha=0.01)
lasso.fit(X_train.reshape(X_train.shape[0], -1), y_train)

# Get Lasso coefficients as additional features
lasso_features_train = lasso.predict(X_train.reshape(X_train.shape[0], -1))
lasso_features_test = lasso.predict(X_test.reshape(X_test.shape[0], -1))
lasso_features_val = lasso.predict(X_val.reshape(X_val.shape[0], -1))

# Reshape Lasso features to be compatible for concatenation
lasso_features_train = lasso_features_train.reshape(-1, 1, 1)
lasso_features_test = lasso_features_test.reshape(-1, 1, 1)
lasso_features_val = lasso_features_val.reshape(-1, 1, 1)

# Concatenate Lasso features with original features
X_train_combined = np.concatenate([X_train, lasso_features_train.repeat(time_step, axis=1)], axis=2)
X_test_combined = np.concatenate([X_test, lasso_features_test.repeat(time_step, axis=1)], axis=2)
X_val_combined = np.concatenate([X_val, lasso_features_val.repeat(time_step, axis=1)], axis=2)


In [104]:
# Define GRU model with L1L2 regularization (mimicking Lasso)
model = Sequential()
model.add(GRU(64, input_shape=(time_step, 2)))  # 2 features: original + Lasso
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')

In [79]:
#Model fitting
model.fit(X_train_combined, y_train, validation_data=(X_test_combined, ytest), epochs=100, batch_size=64, verbose=1)

Epoch 1/100
18/18 [==============================] - 87s 124ms/step - loss: 0.0785 - val_loss: 0.0548
Epoch 2/100
18/18 [==============================] - 2s 122ms/step - loss: 0.0125 - val_loss: 0.0088
Epoch 3/100
18/18 [==============================] - 2s 127ms/step - loss: 0.0059 - val_loss: 0.0110
Epoch 4/100
18/18 [==============================] - 3s 147ms/step - loss: 0.0028 - val_loss: 0.0034
Epoch 5/100
18/18 [==============================] - 3s 170ms/step - loss: 0.0012 - val_loss: 0.0011
Epoch 6/100
18/18 [==============================] - 3s 175ms/step - loss: 5.9257e-04 - val_loss: 4.0362e-04
Epoch 7/100
18/18 [==============================] - 3s 164ms/step - loss: 4.6353e-04 - val_loss: 2.0980e-04
Epoch 8/100
18/18 [==============================] - 2s 82ms/step - loss: 4.6645e-04 - val_loss: 2.0575e-04
Epoch 9/100
18/18 [==============================] - 1s 67ms/step - loss: 4.5108e-04 - val_loss: 2.2342e-04
Epoch 10/100
18/18 [==============================] - 1s 68m

<h1>Dự báo dữ liệu test, val

In [80]:
train_predict = model.predict(X_train_combined)
y_pred = model.predict(X_test_combined)
y_pred_val = model.predict(X_val_combined)


3/3 [==============================] - 0s 21ms/step


<h1>Chuẩn hóa dữ liệu

In [81]:
#Chuẩn hóa dữ liệu y_pred, y_pred_val
train_predict=scaler.inverse_transform(train_predict)
y_pred=scaler.inverse_transform(y_pred)
y_pred_val=scaler.inverse_transform(y_pred_val)

In [82]:
ytest = scaler.inverse_transform(ytest.reshape(-1,1))
ytest

array([[72182.4],
       [71802.5],
       [71232.7],
       [72087.4],
       [69808. ],
       [72087.4],
       [71232.7],
       [70567.8],
       [70472.8],
       [68858.2],
       [68478.3],
       [66673.8],
       [66483.8],
       [67053.7],
       [66008.9],
       [63159.6],
       [62779.7],
       [62494.8],
       [66483.8],
       [66768.7],
       [65534. ],
       [64679.3],
       [62969.7],
       [65439.1],
       [68288.4],
       [67718.5],
       [69333.1],
       [68858.2],
       [68098.4],
       [68288.4],
       [67528.6],
       [66768.7],
       [67243.6],
       [67053.7],
       [67433.6],
       [66673.8],
       [66008.9],
       [63064.6],
       [62779.7],
       [61260.1],
       [64584.3],
       [63349.6],
       [65534. ],
       [69998. ],
       [67053.7],
       [67148.6],
       [67908.5],
       [67528.6],
       [67243.6],
       [68668.3],
       [68573.3],
       [69998. ],
       [71327.6],
       [69428.1],
       [69458.1],
       [71

In [83]:
yval = scaler.inverse_transform(yval.reshape(-1,1))
yval

array([[80100.],
       [80000.],
       [79500.],
       [80000.],
       [79500.],
       [80300.],
       [79100.],
       [79400.],
       [79500.],
       [78900.],
       [78900.],
       [78300.],
       [78000.],
       [77500.],
       [77600.],
       [76100.],
       [76600.],
       [74800.],
       [74200.],
       [75600.],
       [74000.],
       [74400.],
       [72700.],
       [74200.],
       [73700.],
       [74400.],
       [74400.],
       [75500.],
       [75800.],
       [75200.],
       [74800.],
       [73900.],
       [71400.],
       [72300.],
       [70200.],
       [70300.],
       [69000.],
       [67000.],
       [68000.],
       [67600.],
       [68000.],
       [70400.],
       [70900.],
       [70000.],
       [71000.],
       [70800.],
       [71400.],
       [70700.],
       [69600.],
       [68700.],
       [68900.],
       [70700.],
       [70300.],
       [70000.],
       [69300.],
       [69500.],
       [69400.],
       [68700.],
       [68300.

<h1>Đánh giá độ chính xác của thuật toán

In [84]:
# RMSE
valid_rmse = np.sqrt(np.mean((y_pred_val - yval)**2))
test_rmse = np.sqrt(np.mean((y_pred - ytest)**2))
print('Validation RMSE:', valid_rmse)
print('Testing RMSE:', test_rmse)

Validation RMSE: 977.6897968107053
Testing RMSE: 1205.5998539241557


In [85]:
#MAE
valid_mae = mean_absolute_error(yval,y_pred_val)
test_mae = mean_absolute_error(ytest, y_pred)
print('Validation MAE:', valid_mae)
print('Testing MAE:', test_mae)

Validation MAE: 745.9896190068494
Testing MAE: 926.155407914959


In [86]:
#MAPE
valid_mape = np.mean(np.abs((yval - y_pred_val) / yval))*100
test_mape = np.mean(np.abs((ytest - y_pred) / ytest))*100
print('Validation MAPE:', valid_mape)
print('Testing MAPE:', test_mape)

Validation MAPE: 1.0339028118963467
Testing MAPE: 1.2733600601631514


In [87]:
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_1 (GRU)                 (None, 64)                13056     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 13121 (51.25 KB)
Trainable params: 13121 (51.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


<h1>Quá trình dự đoán giá cổ phiếu trong 30 ngày tiếp theo

In [88]:
x_input = val_data[37:].values.reshape(1, -1, 1)

temp_input = list(x_input)
temp_input = temp_input[0].tolist()

lst_output = []
n_steps = 30  # Adjust the sequence length as needed
i = 0

while i < 30:
    if len(temp_input) >= n_steps:
        x_input = np.array(temp_input[-n_steps:])
        x_input = x_input.reshape((1, n_steps, 1))

        # Print for debugging
        print("{} day input {}".format(i, x_input))

        yhat = model.predict(x_input, verbose=0)

        # Print for debugging
        print("{} day output {}".format(i, yhat))

        temp_input.extend(yhat[0].tolist())
        temp_input = temp_input[1:]
        lst_output.extend(yhat.tolist())
        i = i + 1
    else:
        x_input = x_input.reshape((1, len(temp_input), 1))

        # Print for debugging
        print("{} day input {}".format(i, x_input))

        yhat = model.predict(x_input, verbose=0)

        # Print for debugging
        print("{} day output {}".format(i, yhat))

        temp_input.extend(yhat[0].tolist())
        i = i + 1

print(lst_output)



AttributeError: ignored

In [ ]:
print(scaler.inverse_transform(lst_output))

<h1>Vẽ hình mô tả dự đoán

In [ ]:
#Drawing plot
train_data_index = pd.RangeIndex(start=0, stop=train_size, step=1)
plt.plot(scaler.inverse_transform(train_data))
test_data_index = pd.RangeIndex(start=train_size, stop=train_size+test_size, step=1)
plt.plot(test_data_index,scaler.inverse_transform(test_data))
test_data_index = pd.RangeIndex(start=train_size+101, stop=train_size+test_size, step=1)
plt.plot(test_data_index,(y_pred))
val_data_index = pd.RangeIndex(start=train_size+test_size, stop=train_size+test_size+val_size, step=1)
plt.plot(val_data_index,scaler.inverse_transform(val_data))
val_data_index = pd.RangeIndex(start=train_size+test_size+101, stop=train_size+test_size+val_size, step=1)
plt.plot(val_data_index,y_pred_val)
prediect_data_index = pd.RangeIndex(start=len(VNM1)-1, stop=len(VNM1)+29, step=1)
plt.plot(prediect_data_index,scaler.inverse_transform(lst_output))
plt.legend(['Train','Test','Test Predictions ','Validate','Validate Predictions','Next30days Predictions'])
plt.title("Dự đoán giá cổ phiếu VNM trong 30 ngày kế tiếp theo tỉ lệ 7-2-1")
plt.show()

# 5. Chia tập dữ liệu để train-test-validate theo tỉ lệ 6:3:1

In [ ]:
# Split the data into training, testing, and validation sets
train_size = int(0.6 * len(VNM1))
test_size = int(0.3 * len(VNM1))
val_size = len(VNM1) - train_size - test_size

In [ ]:
train_data = VNM1[:train_size]
test_data = VNM1[train_size:train_size+test_size]
val_data = VNM1[train_size+test_size:]

<h1>Hàm Create Dataset

In [ ]:
#Chuyển mảng giá trị của dataset thành ma trận
def create_dataset(dataset, time_step=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-time_step-1):
		a = dataset[i:(i+time_step), 0]   ###i=0, X=0,1,2,3-----99   Y=100
		dataX.append(a)
		dataY.append(dataset[i + time_step, 0])
	return numpy.array(dataX), numpy.array(dataY)

In [ ]:
import numpy
#Reshape into X=t,t+1, t+2... t+99 and Y= t+100
time_step =100
X_train, y_train = create_dataset(train_data, time_step)
X_val, yval = create_dataset(val_data, time_step)
X_test, ytest = create_dataset(test_data, time_step)

In [ ]:
#Reshape input to be [samples, time steps, features] which is required for GRU
X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)
X_val = X_val.reshape(X_val.shape[0],X_val.shape[1] , 1)

In [ ]:
#Define GRU model
model = Sequential()
model.add(GRU(64, input_shape=(time_step, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
#Model fitting
model.fit(X_train,y_train,validation_data=(X_test,ytest),epochs=100,batch_size=64,verbose=1)

<h1>Dự báo dữ liệu test, val

In [ ]:
train_predict=model.predict(X_train)
y_pred=model.predict(X_test)
y_pred_val=model.predict(X_val)

<h1>Chuẩn hóa dữ liệu

In [ ]:
#Chuẩn hóa dữ liệu y_pred, y_pred_val
train_predict=scaler.inverse_transform(train_predict)
y_pred=scaler.inverse_transform(y_pred)
y_pred_val=scaler.inverse_transform(y_pred_val)

In [ ]:
ytest = scaler.inverse_transform(ytest.reshape(-1,1))
ytest

In [ ]:
yval = scaler.inverse_transform(yval.reshape(-1,1))
yval

<h1>Đánh giá độ chính xác của thuật toán

In [ ]:
# RMSE
valid_rmse = np.sqrt(np.mean((y_pred_val - yval)**2))
test_rmse = np.sqrt(np.mean((y_pred - ytest)**2))
print('Validation RMSE:', valid_rmse)
print('Testing RMSE:', test_rmse)

In [ ]:
#MAE
valid_mae = mean_absolute_error(yval,y_pred_val)
test_mae = mean_absolute_error(ytest, y_pred)
print('Validation MAE:', valid_mae)
print('Testing MAE:', test_mae)

In [ ]:
#MAPE
valid_mape = np.mean(np.abs((yval - y_pred_val) / yval))*100
test_mape = np.mean(np.abs((ytest - y_pred) / ytest))*100
print('Validation MAPE:', valid_mape)
print('Testing MAPE:', test_mape)

<h1>Quá trình dự đoán giá cổ phiếu trong 30 ngày tiếp theo

In [ ]:

lst_output = []
n_steps = 30  # Adjust the sequence length as needed
i = 0

while i < 30:
    if len(temp_input) >= n_steps:
        x_input = np.array(temp_input[-n_steps:])
        x_input = x_input.reshape((1, n_steps, 1))

        # Print for debugging
        print("{} day input {}".format(i, x_input))

        yhat = model.predict(x_input, verbose=0)

        # Print for debugging
        print("{} day output {}".format(i, yhat))

        temp_input.extend(yhat[0].tolist())
        temp_input = temp_input[1:]
        lst_output.extend(yhat.tolist())
        i = i + 1
    else:
        x_input = x_input.reshape((1, len(temp_input), 1))

        # Print for debugging
        print("{} day input {}".format(i, x_input))

        yhat = model.predict(x_input, verbose=0)

        # Print for debugging
        print("{} day output {}".format(i, yhat))

        temp_input.extend(yhat[0].tolist())
        i = i + 1

print(lst_output)


In [ ]:
print(scaler.inverse_transform(lst_output))

<h1>Vẽ hình mô tả dự đoán

In [ ]:
#Drawing plot
train_data_index = pd.RangeIndex(start=0, stop=train_size, step=1)
plt.plot(scaler.inverse_transform(train_data))
test_data_index = pd.RangeIndex(start=train_size, stop=train_size+test_size, step=1)
plt.plot(test_data_index,scaler.inverse_transform(test_data))
test_data_index = pd.RangeIndex(start=train_size+101, stop=train_size+test_size, step=1)
plt.plot(test_data_index,(y_pred))
val_data_index = pd.RangeIndex(start=train_size+test_size, stop=train_size+test_size+val_size, step=1)
plt.plot(val_data_index,scaler.inverse_transform(val_data))
val_data_index = pd.RangeIndex(start=train_size+test_size+101, stop=train_size+test_size+val_size, step=1)
plt.plot(val_data_index,y_pred_val)
prediect_data_index = pd.RangeIndex(start=len(VNM1)-1, stop=len(VNM1)+29, step=1)
plt.plot(prediect_data_index,scaler.inverse_transform(lst_output))
plt.legend(['Train','Test','Test Predictions','Validate','Validate Predictions','Next30days Predictions'])
plt.title("Dự đoán giá cổ phiếu VNM trong 30 ngày kế tiếp theo tỉ lệ 6-3-1")
plt.show()

# 6. Chia tập dữ liệu để train-test-validate theo tỉ lệ 5:3:2

In [ ]:
# Split the data into training, testing, and validation sets
train_size = int(0.5 * len(VNM1))
test_size = int(0.3 * len(VNM1))
val_size = len(VNM1) - train_size - test_size

In [ ]:
train_data = VNM1[:train_size]
test_data = VNM1[train_size:train_size+test_size]
val_data = VNM1[train_size+test_size:]

<h1>Hàm Create Dataset

In [ ]:
#Chuyển mảng giá trị của dataset thành ma trận
def create_dataset(dataset, time_step=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-time_step-1):
		a = dataset[i:(i+time_step), 0]   ###i=0, X=0,1,2,3-----99   Y=100
		dataX.append(a)
		dataY.append(dataset[i + time_step, 0])
	return numpy.array(dataX), numpy.array(dataY)

In [ ]:
import numpy
#Reshape into X=t,t+1, t+2... t+99 and Y= t+100
time_step =100
X_train, y_train = create_dataset(train_data, time_step)
X_val, yval = create_dataset(val_data, time_step)
X_test, ytest = create_dataset(test_data, time_step)

In [ ]:
#Reshape input to be [samples, time steps, features] which is required for GRU
X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)
X_val = X_val.reshape(X_val.shape[0],X_val.shape[1] , 1)

In [ ]:
#Define GRU model
model = Sequential()
model.add(GRU(64, input_shape=(time_step, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
#Model fitting
model.fit(X_train,y_train,validation_data=(X_test,ytest),epochs=100,batch_size=64,verbose=1)

<h1>Dự báo dữ liệu test, val

In [ ]:
train_predict=model.predict(X_train)
y_pred=model.predict(X_test)
y_pred_val=model.predict(X_val)

<h1>Chuẩn hóa dữ liệu

In [ ]:
#Chuẩn hóa dữ liệu y_pred, y_pred_val
train_predict=scaler.inverse_transform(train_predict)
y_pred=scaler.inverse_transform(y_pred)
y_pred_val=scaler.inverse_transform(y_pred_val)

In [ ]:
ytest = scaler.inverse_transform(ytest.reshape(-1,1))
ytest

In [ ]:
yval = scaler.inverse_transform(yval.reshape(-1,1))
yval

<h1>Đánh giá độ chính xác của thuật toán

In [ ]:
# RMSE
valid_rmse = np.sqrt(np.mean((y_pred_val - yval)**2))
test_rmse = np.sqrt(np.mean((y_pred - ytest)**2))
print('Validation RMSE:', valid_rmse)
print('Testing RMSE:', test_rmse)

In [ ]:
#MAE
valid_mae = mean_absolute_error(yval,y_pred_val)
test_mae = mean_absolute_error(ytest, y_pred)
print('Validation MAE:', valid_mae)
print('Testing MAE:', test_mae)

In [ ]:
#MAPE
valid_mape = np.mean(np.abs((yval - y_pred_val) / yval))*100
test_mape = np.mean(np.abs((ytest - y_pred) / ytest))*100
print('Validation MAPE:', valid_mape)
print('Testing MAPE:', test_mape)

<h1>Quá trình dự đoán giá cổ phiếu trong 30 ngày tiếp theo

In [ ]:

lst_output = []
n_steps = 30  # Adjust the sequence length as needed
i = 0

while i < 30:
    if len(temp_input) >= n_steps:
        x_input = np.array(temp_input[-n_steps:])
        x_input = x_input.reshape((1, n_steps, 1))

        # Print for debugging
        print("{} day input {}".format(i, x_input))

        yhat = model.predict(x_input, verbose=0)

        # Print for debugging
        print("{} day output {}".format(i, yhat))

        temp_input.extend(yhat[0].tolist())
        temp_input = temp_input[1:]
        lst_output.extend(yhat.tolist())
        i = i + 1
    else:
        x_input = x_input.reshape((1, len(temp_input), 1))

        # Print for debugging
        print("{} day input {}".format(i, x_input))

        yhat = model.predict(x_input, verbose=0)

        # Print for debugging
        print("{} day output {}".format(i, yhat))

        temp_input.extend(yhat[0].tolist())
        i = i + 1

print(lst_output)


In [ ]:
print(scaler.inverse_transform(lst_output))

<h1>Vẽ hình mô tả dự đoán

In [ ]:
#Drawing plot
train_data_index = pd.RangeIndex(start=0, stop=train_size, step=1)
plt.plot(scaler.inverse_transform(train_data))
test_data_index = pd.RangeIndex(start=train_size, stop=train_size+test_size, step=1)
plt.plot(test_data_index,scaler.inverse_transform(test_data))
test_data_index = pd.RangeIndex(start=train_size+101, stop=train_size+test_size, step=1)
plt.plot(test_data_index,(y_pred))
val_data_index = pd.RangeIndex(start=train_size+test_size, stop=train_size+test_size+val_size, step=1)
plt.plot(val_data_index,scaler.inverse_transform(val_data))
val_data_index = pd.RangeIndex(start=train_size+test_size+101, stop=train_size+test_size+val_size, step=1)
plt.plot(val_data_index,y_pred_val)
prediect_data_index = pd.RangeIndex(start=len(VNM1)-1, stop=len(VNM1)+29, step=1)
plt.plot(prediect_data_index,scaler.inverse_transform(lst_output))
plt.legend(['Train','Test','Test Predictions','Validate','Validate Predictions','Next30days Predictions'])
plt.title("Dự đoán giá cổ phiếu VNM trong 30 ngày kế tiếp theo tỉ lệ 5-3-2")
plt.show()